In [ ]:
!pip install xgboost
!pip install LGBMRegressor
!pip install CatBoostRegressor

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement LGBMRegressor (from versions: none)
ERROR: No matching distribution found for LGBMRegressor
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement CatBoostRegressor (from versions: none)
ERROR: No matching distribution found for CatBoostRegressor
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
!pip install catboost



You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
data = pd.read_csv('/work/train_hack4bengal.csv')


In [ ]:
# 'Gender', 'Company Type', 'WFH Setup Available', 'Designation',
#        'Resource Allocation', 'Mental Fatigue Score'

# ['Male','Service','Yes','3','2','5.5']

In [ ]:
data

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,fffe32003000360033003200,2008-09-30,Female,Service,No,2.0,3.0,3.8,0.16
1,fffe3700360033003500,2008-11-30,Male,Service,Yes,1.0,2.0,5.0,0.36
2,fffe31003300320037003900,2008-03-10,Female,Product,Yes,2.0,NaN,5.8,0.49
3,fffe32003400380032003900,2008-11-03,Male,Service,Yes,1.0,1.0,2.6,0.20
4,fffe31003900340031003600,2008-07-24,Female,Service,No,3.0,7.0,6.9,0.52
...,...,...,...,...,...,...,...,...,...
22745,fffe31003500370039003100,2008-12-30,Female,Service,No,1.0,3.0,NaN,0.41
22746,fffe33003000350031003800,2008-01-19,Female,Product,Yes,3.0,6.0,6.7,0.59
22747,fffe390032003000,2008-11-05,Male,Service,Yes,3.0,7.0,NaN,0.72
22748,fffe33003300320036003900,2008-01-10,Female,Service,No,2.0,5.0,5.9,0.52


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22750 entries, 0 to 22749
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Employee ID           22750 non-null  object 
 1   Date of Joining       22750 non-null  object 
 2   Gender                22750 non-null  object 
 3   Company Type          22750 non-null  object 
 4   WFH Setup Available   22750 non-null  object 
 5   Designation           22750 non-null  float64
 6   Resource Allocation   21369 non-null  float64
 7   Mental Fatigue Score  20633 non-null  float64
 8   Burn Rate             21626 non-null  float64
dtypes: float64(4), object(5)
memory usage: 1.6+ MB


In [ ]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop Employee ID column
    df = df.drop('Employee ID', axis=1)

    # Drop Joining Date
    df = df.drop('Date of Joining', axis=1)
    
    # Drop rows with missing target values
    missing_target_rows = df.loc[df['Burn Rate'].isna(), :].index
    df = df.drop(missing_target_rows, axis=0).reset_index(drop=True)
    
    # Fill remaining missing values with column means
    for column in ['Resource Allocation', 'Mental Fatigue Score']:
        df[column] = df[column].fillna(df[column].mean())
    
    
    # Binary encoding
    df['Gender'] = df['Gender'].replace({'Female': 0, 'Male': 1})
    df['Company Type'] = df['Company Type'].replace({'Product': 0, 'Service': 1})
    df['WFH Setup Available'] = df['WFH Setup Available'].replace({'No': 0, 'Yes': 1})
    
    # Split df into X and y
    y = df['Burn Rate']
    X = df.drop('Burn Rate', axis=1)
     # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [ ]:
X_train.columns

Index(['Gender', 'Company Type', 'WFH Setup Available', 'Designation',
       'Resource Allocation', 'Mental Fatigue Score'],
      dtype='object')

In [ ]:
models = {
    "                     Linear Regression": LinearRegression(),
    " Linear Regression (L2 Regularization)": Ridge(),
    " Linear Regression (L1 Regularization)": Lasso(),
    "                   K-Nearest Neighbors": KNeighborsRegressor(),
    "                        Neural Network": MLPRegressor(),
    "Support Vector Machine (Linear Kernel)": LinearSVR(),
    "   Support Vector Machine (RBF Kernel)": SVR(),
    "                         Decision Tree": DecisionTreeRegressor(),
    "                         Random Forest": RandomForestRegressor(),
    "                     Gradient Boosting": GradientBoostingRegressor(),
    "                               XGBoost": XGBRegressor(),
    "                              LightGBM": LGBMRegressor(),
    "                              CatBoost": CatBoostRegressor(verbose=0)
}

for name, model in models.items():
    model.fit(X_train.values, y_train.values)
    print(name + " trained.")

                     Linear Regression trained.
 Linear Regression (L2 Regularization) trained.
 Linear Regression (L1 Regularization) trained.
                   K-Nearest Neighbors trained.
                        Neural Network trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                         Decision Tree trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                               XGBoost trained.
                              LightGBM trained.
                              CatBoost trained.


In [ ]:
for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_test.values, y_test.values)))

                     Linear Regression R^2 Score: 0.87075
 Linear Regression (L2 Regularization) R^2 Score: 0.87075
 Linear Regression (L1 Regularization) R^2 Score: -0.00001
                   K-Nearest Neighbors R^2 Score: 0.88506
                        Neural Network R^2 Score: 0.88481
Support Vector Machine (Linear Kernel) R^2 Score: 0.86896
   Support Vector Machine (RBF Kernel) R^2 Score: 0.88891
                         Decision Tree R^2 Score: 0.87770
                         Random Forest R^2 Score: 0.89201
                     Gradient Boosting R^2 Score: 0.90341
                               XGBoost R^2 Score: 0.90684
                              LightGBM R^2 Score: 0.90998
                              CatBoost R^2 Score: 0.91015


In [ ]:
import pickle

In [ ]:
for name, model in models.items():
    filename = '/work/Models/'+name+'.pkl'
    pickle.dump(model, open(filename, 'wb'))

In [ ]:
import pickle

#
# Create your model here (same as above)
#

# Save to file in the current working directory
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

In [ ]:
# with open(pkl_filename, 'rb') as file:
# pickle_model = pickle.load("/work/Models/XGBoost.pkl")

with open("/work/Models/                               XGBoost.pkl", 'rb') as file:
    pickle_model = pickle.load(file)

In [ ]:
X_test

,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score
9862,-0.954022,0.725052,0.919713,-1.034140,-1.743017,-1.329772
8996,1.048194,0.725052,0.919713,0.725025,0.768001,0.201658
19493,-0.954022,0.725052,0.919713,-1.034140,-0.236406,0.092271
8487,-0.954022,0.725052,0.919713,-1.034140,-0.738610,-0.399975
11803,-0.954022,-1.379211,0.919713,-1.034140,-1.240814,-1.001608
...,...,...,...,...,...,...
12470,1.048194,-1.379211,-1.087295,0.725025,0.265797,-0.235893
9959,1.048194,0.725052,0.919713,-0.154557,-0.738610,-0.837526
18811,-0.954022,0.725052,0.919713,0.725025,0.265797,-0.399975
2060,-0.954022,0.725052,-1.087295,2.484190,2.274612,2.225334


In [ ]:
def encode(arr):
    b = []
    for i in range(len(arr)):
        if(arr[i] == 'Male'):
            b.append(1)
        elif(arr[i] == 'Female'):
            b.append(0)
        elif(arr[i] == 'Product'):
            b.append(0)
        elif(arr[i] == 'Service'):
            b.append(1)
        elif(arr[i] == 'No'):
            b.append(0)
        elif(arr[i] == 'Yes'):
            b.append(1)           
        else:
            b.append(a[i])
    return b


In [ ]:
a = ['Male','Service','Yes', 3, 2, 5.5]
a = encode(a)
arr = np.array([a])
b = arr
b

array([[1. , 1. , 1. , 3. , 2. , 5.5]])

In [ ]:
Ypredict = pickle_model.predict(b)

In [ ]:
Ypredict 

array([0.9177508], dtype=float32)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=167ceb4c-421d-4f13-bafe-633219b94dfe' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>